In [1]:
import sys
import os
import csv
import re
import codecs
import numpy as np
from langua import Predict
from pandas import read_csv
from pandas import Series
from keras import layers
from keras.models import Model
from keras import initializers
from keras import layers
from keras import constraints
from keras import regularizers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Embedding
from keras import optimizers

C:\Users\vkhanna\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_data():
    # Read the data
    train_data_raw = read_csv("./data/train.csv");
    test_data_raw = read_csv("./data/test.csv")
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    y_train = train_data_raw[labels].values
    x_train = train_data_raw["comment_text"]
    x_test = test_data_raw["comment_text"]
    output = read_csv('data/sample_submission.csv')
    return x_train, y_train, x_test

x_train, y_train, x_test = read_data()

In [3]:
# remove data points which are not english
def translate_input():
    input_file_path = './data/train.csv'
    output_file_path = './data/train_translated.csv'
    data = read_csv(input_file_path)
    data['lang'] = Series('en', index=data.index)
    p = Predict()
    for index, point in data.iterrows():
        comment = point['comment_text']
        try:
            language_detected = p.get_lang(comment)
            if language_detected != 'en':
                data.set_value(index, 'lang', 'non_en')
        except:
            data.set_value(index, 'lang', 'non_en')
    data = data[data.lang != 'non_en']
    data.to_csv(output_file_path, index=None)

In [ ]:
translate_input()

C:\Users\vkhanna\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\vkhanna\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [6]:
dimensions = 100

# get the embedding from glove file 
pretrained_vectors = {}
f = open('./data/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.rstrip().rsplit(' ', dimensions)
    word = values[0]
    probabilities = np.asarray(values[1:], dtype='float32')
    pretrained_vectors[word] = probabilities
f.close()

print('Found {} word vectors.'.format(len(pretrained_vectors)))

Found 400000 word vectors.


In [3]:
dimensions = 300

# Get embeddings
pretrained_vectors = {}
f = open('data/wiki.en.vec', encoding="utf8")
for line in f:
    values = line.rstrip().rsplit(' ', dimensions)
    word = values[0]
    probabilities = np.asarray(values[1:], dtype='float32')
    pretrained_vectors[word] = probabilities
f.close()

print('Found {} word vectors.'.format(len(pretrained_vectors)))

Found 2519371 word vectors.


In [ ]:
# following are sample words in the glove embeddings
print(pretrained_vectors['kill'])
print(pretrained_vectors['murder'])

In [4]:
#Read the data
train_data_raw =  read_csv("./data/train_translated.csv");
test_data_raw =  read_csv("./data/test.csv")

labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train_data_raw[labels].values
x_train = train_data_raw["comment_text"]
x_test  = test_data_raw["comment_text"]

In [5]:
train_data = read_csv('./data/train.csv')

In [6]:
len(train_data) - len(train_data_raw)

4341

In [7]:
tknzr = Tokenizer(char_level=False, oov_token=None, num_words=None, split=' ', lower=True, 
                      filters='!"#$%&()*+,\t-./:;\n<=>?@[\]^_`{|}~ ', document_count=0)

list_x_train = list(x_train)
tknzr.fit_on_texts(list_x_train)

train_seq = tknzr.texts_to_sequences(x_train)
test_seq = tknzr.texts_to_sequences(x_test)

In [8]:
# max comment length
print(max(len(sequence) for sequence in train_seq))
# fixing the max comment size to cover 90% of the comments 
processing_length = int(np.percentile([len(sequence) for sequence in train_seq], 90))
print(processing_length)

1404
160


In [9]:
padded_X_train = pad_sequences(train_seq, maxlen=processing_length, padding='post', value=0.0, truncating='post')
padded_X_test = pad_sequences(test_seq, maxlen=processing_length, padding='post', value=0.0, truncating='post')

In [10]:
for index, (token, token_index) in enumerate(zip(x_train[60:61], train_seq[60:61])):
    print('Comment:  {}'.format(token))
    print('Tokenized Comment: {}'.format(token_index))

Comment:  A Bisexual, like a homosexual or a heterosexual, is not defined by sexual activity. (Much like a 15 year old boy who is attracted to a girl sexually but has never had sex is still straight). A person who is actually sexually attracted/aroused by the same sex as well as the opposite sex is bisexual.
Tokenized Comment: [6, 10921, 51, 6, 3298, 26, 6, 9808, 9, 15, 1763, 33, 1951, 2185, 132, 51, 6, 666, 405, 349, 1671, 63, 9, 8172, 3, 6, 1995, 7786, 27, 44, 194, 100, 1402, 9, 156, 1683, 6, 221, 63, 9, 208, 7786, 8172, 22207, 33, 1, 139, 1402, 18, 101, 18, 1, 2087, 1402, 9, 10921]


In [11]:
def get_embedding_matrix():
    embed_matrix = np.zeros((len(tknzr.word_index) + 1, dimensions))
    for word, index in tknzr.word_index.items():
        vector = pretrained_vectors.get(word)
        if vector is not None:
            embed_matrix[index] = vector
    return embed_matrix

embd_matrix = get_embedding_matrix()

In [12]:
index = tknzr.word_index['kill']
print(pretrained_vectors['kill']) 
print(embd_matrix[index])

[-0.17406   -0.094699  -0.44253    0.10427   -0.28       0.37218
 -0.02919    0.047345   0.0097075  0.2402    -0.20904    0.21263
 -0.16953   -0.22599    0.030993  -0.14778    0.012086  -0.24262
 -0.10528   -0.14844   -0.081931   0.11868   -0.5627    -0.34811
 -0.087365  -0.08099   -0.15424   -0.015826  -0.22876    0.33089
 -0.037312   0.28776   -0.2827    -0.01459    0.13523   -0.14421
  0.23173   -0.43755    0.061313  -0.26361    0.32181    0.1504
  0.1521     0.17965    0.13317    0.056412  -0.0087446  0.13166
 -0.008926   0.037683   0.0090371 -0.0081518  0.13471   -0.071495
 -0.32246    0.22833    0.1576     0.087777  -0.27286    0.15595
 -0.52822    0.21661    0.050134   0.46708   -0.10385    0.15519
 -0.055313   0.42306    0.087317  -0.043844   0.29568   -0.056677
 -0.13641   -0.49272    0.072833   0.017133   0.041882  -0.11428
 -0.13504   -0.013326   0.5042    -0.17995   -0.034001   0.15967
 -0.1254     0.35338   -0.053087  -0.025029   0.10062   -0.22975
  0.096282  -0.038608  -

In [29]:
from keras.layers import GlobalMaxPooling1D

model = Sequential()
model.add(Embedding(len(tknzr.word_index) + 1, dimensions, weights=[embd_matrix], input_length=processing_length, trainable=False))

model.add(Dropout(0.2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(6, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 160, 300)          65599500  
_________________________________________________________________
dropout_3 (Dropout)          (None, 160, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 156, 128)          192128    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 60)                7740      
__________

In [30]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Nadam(lr=0.001), metrics=['binary_accuracy'])

In [31]:
[X_final_train, X_final_test, Y_final_train, Y_final_test] = train_test_split(padded_X_train, y_train, test_size=0.10, shuffle=False)



In [33]:
graph = model.fit(X_final_train, Y_final_train, batch_size=64, epochs=6,
                  validation_data=(X_final_test, Y_final_test), verbose=1, shuffle=False)

Train on 139707 samples, validate on 15523 samples
Epoch 1/6
139707/139707 [==============================] - 505s 4ms/step - loss: 0.0677 - binary_accuracy: 0.9760 - val_loss: 0.0466 - val_binary_accuracy: 0.9827
Epoch 2/6
139707/139707 [==============================] - 506s 4ms/step - loss: 0.0454 - binary_accuracy: 0.9827 - val_loss: 0.0457 - val_binary_accuracy: 0.9826
Epoch 3/6
139707/139707 [==============================] - 505s 4ms/step - loss: 0.0415 - binary_accuracy: 0.9840 - val_loss: 0.0452 - val_binary_accuracy: 0.9831
Epoch 4/6
139707/139707 [==============================] - 505s 4ms/step - loss: 0.0383 - binary_accuracy: 0.9848 - val_loss: 0.0485 - val_binary_accuracy: 0.9829
Epoch 5/6
139707/139707 [==============================] - 504s 4ms/step - loss: 0.0356 - binary_accuracy: 0.9856 - val_loss: 0.0479 - val_binary_accuracy: 0.9830
Epoch 6/6
139707/139707 [==============================] - 504s 4ms/step - loss: 0.0337 - binary_accuracy: 0.9863 - val_loss: 0.0525 -

In [34]:
model.save("detoxit_model_sigmoid.h5")

In [18]:
from keras.models import load_model

new_model = load_model("detoxit_model.h5")

In [19]:
result = new_model.predict(padded_X_test, batch_size=32, verbose=0, steps= None)


In [44]:
submission_input = read_csv("./data/sample_submission.csv")
submission_input[labels] = result
submission_input.to_csv("submission_file_30_oct_cnn_fasttext.csv", index=False)

In [37]:
string = 'fuck you bitches'
new_string = [string]
new_string = tknzr.texts_to_sequences(new_string)
print(new_string)
new_string = pad_sequences(new_string, maxlen=processing_length, padding='post', truncating='post')

# Predict
prediction = model.predict(new_string)

[[193, 8, 7646]]


In [25]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tknzr, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
prediction

array([[4.3697548e-01, 4.0647509e-03, 3.8161206e-01, 4.9847891e-05,
        1.6307604e-01, 1.4221812e-02]], dtype=float32)

In [38]:
prediction
# Print output
print("Toxicity levels for '{}':".format(string))
print('Toxic:         {:.0%}'.format(prediction[0][0]))
print('Severe Toxic:  {:.0%}'.format(prediction[0][1]))
print('Obscene:       {:.0%}'.format(prediction[0][2]))
print('Threat:        {:.0%}'.format(prediction[0][3]))
print('Insult:        {:.0%}'.format(prediction[0][4]))
print('Identity Hate: {:.0%}'.format(prediction[0][5]))
print()

Toxicity levels for 'fuck you bitches':
Toxic:         100%
Severe Toxic:  69%
Obscene:       100%
Threat:        0%
Insult:        99%
Identity Hate: 14%



In [93]:
lstm_model = Sequential()
lstm_model.add(Embedding(len(tknzr.word_index) + 1, dimensions, weights=[embd_matrix], input_length=processing_length, trainable=False))

lstm_model.add(LSTM(2))
lstm_model.add(Dropout(0.5))
lstm_model.add(layers.Dense(6, activation='sigmoid'))
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 156, 300)          62335500  
_________________________________________________________________
lstm_8 (LSTM)                (None, 2)                 2424      
_________________________________________________________________
dropout_13 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 18        
Total params: 62,337,942
Trainable params: 2,442
Non-trainable params: 62,335,500
_________________________________________________________________


In [94]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy'])
[X_final_train, X_final_test, Y_final_train, Y_final_test] = train_test_split(padded_X_train, y_train, test_size=0.10, shuffle=False)

In [95]:
graph = model.fit(X_final_train, Y_final_train, batch_size=64, epochs=1,
                  validation_data=(X_final_test, Y_final_test), verbose=1, shuffle=False)

Train on 139738 samples, validate on 15527 samples
Epoch 1/1
139738/139738 [==============================] - 172s 1ms/step - loss: 0.0457 - acc: 0.9827 - val_loss: 0.0509 - val_acc: 0.9817


In [96]:
result = model.predict(padded_X_test, batch_size=32, verbose=0, steps= None)

In [97]:
submission_input = read_csv("data/sample_submission.csv")
submission_input[labels] = result
submission_input.to_csv("submission_file_21_nov_lstm_6.csv", index=False)